In [1]:
import numpy as np
import pandas as pd
import pickle
from skimage.io import imread, imsave
from skimage import img_as_ubyte
from matplotlib import pyplot as plt
from collections import Counter

from platform import python_version

%matplotlib inline

np.__version__, python_version()

('1.18.5', '3.8.3')

### gaussian_kernel

In [43]:
def gaussian_kernel(size, sigma):
    """
    Построение ядра фильтра Гаусса.

    @param  size  int    размер фильтра (нечетный)
    @param  sigma float  параметр размытия
    @return numpy array  фильтр Гаусса размером size x size
    """
    center = size // 2
    x_grid, y_grid = np.mgrid[:size, :size]
    dists = np.sqrt((x_grid - center)**2 + (y_grid - center)**2)
    gaussian_filter = 1 / (2 * np.pi * sigma**2) * np.exp(- dists**2 / (2 * sigma**2))
    gaussian_filter = gaussian_filter / np.sum(gaussian_filter)
    return gaussian_filter

In [44]:
gaussian_kernel(size=5, sigma=1)

array([[0.00296902, 0.01330621, 0.02193823, 0.01330621, 0.00296902],
       [0.01330621, 0.0596343 , 0.09832033, 0.0596343 , 0.01330621],
       [0.02193823, 0.09832033, 0.16210282, 0.09832033, 0.02193823],
       [0.01330621, 0.0596343 , 0.09832033, 0.0596343 , 0.01330621],
       [0.00296902, 0.01330621, 0.02193823, 0.01330621, 0.00296902]])

### fourier_transform

In [154]:
from scipy.fft import fft2

def fourier_transform(h, shape):
    """
    Получение Фурье-образа искажающей функции

    @param  h            numpy array  искажающая функция h (ядро свертки)
    @param  shape        list         требуемый размер образа
    @return numpy array  H            Фурье-образ искажающей функции h
    """
    return fft2(h, s=shape)

In [155]:
fourier_transform([[1]], [3, 3])

array([[1.-0.j, 1.+0.j, 1.-0.j],
       [1.+0.j, 1.+0.j, 1.-0.j],
       [1.-0.j, 1.+0.j, 1.-0.j]])

### inverse_kernel

In [177]:
def inverse_kernel(H, threshold=1e-10):
    """
    Получение H_inv

    @param  H            numpy array    Фурье-образ искажающей функции h
    @param  threshold    float          порог отсечения для избежания деления на 0
    @return numpy array  H_inv
    """
    H_inv = np.zeros(H.shape, dtype=np.complex64)
    mask = (np.abs(H) > threshold)
    H_inv[mask] = 1 / H[mask]
    return H_inv

In [178]:
inverse_kernel(np.array([
    [10.,   -10.j, 100, 2.j],
    [2.j,   5.j,   0,   100.],
    [100.j, 1.j,   10., 5. - 10.j]
]), threshold=5)

array([[ 0.1 +0.j  , -0.  +0.1j ,  0.01+0.j  ,  0.  +0.j  ],
       [ 0.  +0.j  ,  0.  +0.j  ,  0.  +0.j  ,  0.01+0.j  ],
       [ 0.  -0.01j,  0.  +0.j  ,  0.1 +0.j  ,  0.04+0.08j]],
      dtype=complex64)

### inverse_filtering

In [242]:
from scipy.fft import ifft2

def inverse_filtering(blurred_img, h, threshold=1e-10):
    """
    Метод инверсной фильтрации

    @param  blurred_img    numpy array  искаженное изображение
    @param  h              numpy array  искажающая функция
    @param  threshold      float        параметр получения H_inv
    @return numpy array                 восстановленное изображение
    """
    G = fourier_transform(blurred_img, blurred_img.shape)
    H = fourier_transform(h, G.shape)
    H_inv = inverse_kernel(H, threshold)
    F = G * H_inv
    f = np.abs(ifft2(F))
    return f

In [245]:
inverse_filtering(np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]), np.array([[1]]), threshold=0)

array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]])

### wiener_filtering

In [252]:
def wiener_filtering(blurred_img, h, K=0):
    """
    Винеровская фильтрация

    @param  blurred_img    numpy array  искаженное изображение
    @param  h              numpy array  искажающая функция
    @param  K              float        константа из выражения (8)
    @return numpy array                 восстановленное изображение
    """
    G = fourier_transform(blurred_img, blurred_img.shape)
    H = fourier_transform(h, G.shape)
    H_conj = np.conjugate(H)
    H_square = H_conj * H
    F = H_conj / (H_square + K) * G
    f = np.abs(ifft2(F))
    return f

In [253]:
wiener_filtering(np.array([
        [0.4, 0.4, 0.6],
        [0.4, 0.4, 0.6],
        [0.6, 0.6, 1.]
    ]), np.array([
        [0,   0.2, 0],
        [0.2, 0.2, 0.2],
        [0,   0.2, 0]
    ]))

array([[1.11022302e-16, 1.00000000e+00, 1.11022302e-16],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.66533454e-16, 1.00000000e+00, 1.66533454e-16]])

### compute_psnr

In [292]:
def compute_psnr(img1, img2):
    """
    PSNR metric

    @param  img1    numpy array   оригинальное изображение
    @param  img2    numpy array   искаженное изображение
    @return float   PSNR(img1, img2)
    """
    h, w = img1.shape
    MSE = np.sum((img1 - img2)**2) / (h*w)
    PSNR = 20 * np.log10(255 / np.sqrt(MSE))
    return PSNR

In [293]:
compute_psnr(np.array([
        [1, 2, 3],
        [4, 0, 6],
        [7, 8, 9]
    ]), np.array([
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]
    ]))

43.69382861635198

### optimal K for wiener_filtering

In [299]:
original_img = np.load('tests/06_unittest_filtering_constant_input/original_img.npy')
noisy_img = np.load('tests/06_unittest_filtering_constant_input/noisy_img.npy')

kernel = gaussian_kernel(size=15, sigma=5)

In [316]:
res = []
for K in np.linspace(0,0.01,100):
    filtered_img = wiener_filtering(noisy_img, kernel, K=K)
    res.append((K, compute_psnr(filtered_img, original_img) - compute_psnr(noisy_img, original_img)))

In [317]:
sorted(res, key=lambda x: x[1], reverse=True)[:5]

[(0.00010101010101010101, 10.24266167903923),
 (0.00020202020202020202, 9.759180476815132),
 (0.00030303030303030303, 9.36357855058651),
 (0.00040404040404040404, 9.047576858179283),
 (0.000505050505050505, 8.78752490778686)]